# BLAST

This example illustrates how to perform meta-blocking by using BLAST.

BLAST can be only used on clean-clean datasets (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [3]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 7.0 MB/s eta 0:00:012
     |████████████████████████████████| 198 kB 12.5 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3569006b21628eca4e103e7c64562fe10ce92d74bd35e7a3af49628bce082d76
  Stored in directory: /home/aditi/.cache/pip/wheels/52/45/50/69db7b6e1da74a1b9fcc097827db9185cb8627117de852731e
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyspark
import sparker
import random
import pandas as pd

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

2022-04-10 00:29:32,861 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-04-10 00:29:32,862 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-10 00:29:33,213 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [149]:
df1 = ss.read.json('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/data_1/data/1.json')


In [150]:
df2 = ss.read.json('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/data_1/data/4.json')


In [151]:
df1 = df1.drop("AboutMe")
df1.show(5)

+----------+----+-------+--------------------+---+-------------+--------------+--------------------+
| Birthdate|City|Country|         DisplayName| ID|  PhoneNumber|         State|             address|
+----------+----+-------+--------------------+---+-------------+--------------+--------------------+
|2000-04-20|null|  India|    Dolly Chaurasiya|  0|+917703901236|Andhra Pradesh|Ap #324-5397 Semp...|
|1990-09-03|null|  India|Patnaik Priyadars...|  1|+918939498995|Andhra Pradesh|Opposite Winterco...|
|1993-03-20|null|  India|   Sushaant Valmikei|  2|+917881007998|Andhra Pradesh|P.O. Box 861,Near...|
|1978-10-03|null|  India|        Shiva Sonkar|  3|+919910914898|Andhra Pradesh|P.O. Box 801, 969...|
|1970-06-15|null|  India|     Janardan Desuza|  4|+918839083491|Andhra Pradesh|P.O. Box 541, 912...|
+----------+----+-------+--------------------+---+-------------+--------------+--------------------+
only showing top 5 rows



In [152]:
df1.write.json("file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/data_1/data/output/1.json", mode="overwrite")

In [153]:
df2 = df2.drop("AboutMe")
df2.show(5)

+----------+-----+-------+----------------+-----------+-------------+----------------+--------------------+
| Birthdate| City|Country|     DisplayName|         ID|  PhoneNumber|           State|             address|
+----------+-----+-------+----------------+-----------+-------------+----------------+--------------------+
|2001-12-14| null|  India|Mahua Jahagirdar|25769803776|+917820716007|Himachal Pradesh|Bagpashog,SIRMAUR...|
|1994-11-23| null|  India|       Kiara Rao|25769803777|+915739246356|Himachal Pradesh|Bagthan,SIRMAUR,H...|
|1996-05-20|Solan|  India|  Poonam Runthla|25769803778|+918924350967|Himachal Pradesh|Baila,SOLAN,Himac...|
|1990-07-23|Solan|  India|       Gomed Som|25769803779|+918070753226|Himachal Pradesh|Bakhalag,SOLAN,Hi...|
|1971-04-04| null|  India|  Apoorv Prakash|25769803780|+916543870600|Himachal Pradesh|Bakras,SIRMAUR,Hi...|
+----------+-----+-------+----------------+-----------+-------------+----------------+--------------------+
only showing top 5 rows



In [154]:
df2.write.json("file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/data_1/data/output/2.json", mode="overwrite")

## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.
*source_id* is used to identify from which dataset the profile belongs, it is necessary for the attributes alignment

In [95]:
# # Profiles contained in the first dataset
# profiles1 = sparker.CSVWrapper.load_profiles('/Final_Project/sample/Sample1_cleaned.csv', 
#                                               real_id_field = "id", 
#                                               source_id=1)
# # Max profile id in the first dataset, used to separate the profiles in the next phases
# separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# # Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# # could work with multiple datasets
# separator_ids = [separator_id]

In [111]:
# Profiles contained in the first dataset
profiles1 = sparker.JSONWrapper.load_profiles(file_path = 'file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/data_1/data/output/1.json',
                                             start_id_from = 0,
                                             real_id_field = "ID",
                                            source_id = 0)
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

In [112]:
separator_ids

[13143]

Let's visualize a profile to check if they are correctly loaded

In [113]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'Birthdate', 'value': '2000-04-20'}, {'key': 'Country', 'value': 'India'}, {'key': 'DisplayName', 'value': 'Dolly Chaurasiya'}, {'key': 'PhoneNumber', 'value': '+917703901236'}, {'key': 'State', 'value': 'Andhra Pradesh'}, {'key': 'address', 'value': 'Ap #324-5397 Semper Av.,Opposite Wildemarsh,A Narayanapuram,ANANTHAPUR,Andhra Pradesh,515004'}], 'original_id': '0', 'source_id': 0}


Loads the second dataset and extract the max id (it will be used later)

In [99]:
# profiles2 = sparker.CSVWrapper.load_profiles('/Final_Project/sample/Sample2_cleaned.csv', 
#                                               start_id_from = separator_id+1, 
#                                               real_id_field = "id", 
#                                               source_id=2)
# # Max profile id
# max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

In [115]:
profiles2 = sparker.JSONWrapper.load_profiles(file_path = 'file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/data_1/data/output/2.json',
                                              start_id_from = separator_id+1,
                                              real_id_field = "ID",
                                              source_id = 1)
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

Finally, concatenate the two RDDs

In [116]:
profiles = profiles1.union(profiles2)

In [117]:
profiles.toDF().show(4590) #b000jz4hqo

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{Birthdate, 2000...|          0|         0|        0|
|[{Birthdate, 1990...|          1|         1|        0|
|[{Birthdate, 1993...|          2|         2|        0|
|[{Birthdate, 1978...|          3|         3|        0|
|[{Birthdate, 1970...|          4|         4|        0|
|[{Birthdate, 1987...|          5|         5|        0|
|[{Birthdate, 1997...|          6|         6|        0|
|[{Birthdate, 1983...|          7|         7|        0|
|[{Birthdate, 1997...|          8|         8|        0|
|[{Birthdate, 1986...|          9|         9|        0|
|[{Birthdate, 1976...|         10|        10|        0|
|[{Birthdate, 1971...|         11|        11|        0|
|[{Birthdate, 1991...|         12|        12|        0|
|[{Birthdate, 1975...|         13|        13|        0|
|[{Birthdate, 1982...|         14|        14|   

### Groundtruth (optional)
If you have a groundtruth you can measure the performance at each blocking step.

When you load the groundtruth, it contains the original profiles IDs, so it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [118]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
# gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/sample/ground_truth_sample.csv', 'id1', 'id2')

In [119]:
gt.toDF().show(1400)

+---------------+----------------+
|first_entity_id|second_entity_id|
+---------------+----------------+
|             20|               0|
|             21|               1|
|             22|               2|
|             23|               3|
|             24|               4|
|             25|               5|
|             26|               6|
|             27|               7|
|             28|               8|
|             29|               9|
+---------------+----------------+



In [120]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [121]:
new_gt

{(20, 21670),
 (21, 21671),
 (22, 21672),
 (23, 21673),
 (24, 21674),
 (25, 21675),
 (26, 21676),
 (27, 21677),
 (28, 21678),
 (29, 21679)}

In [122]:
# We can explore some pairs
random.sample(new_gt, 2)

/tmp/ipykernel_5828/3362377442.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(new_gt, 2)


[(22, 21672), (24, 21674)]

## Attributes alignment

BLAST employs LSH to automatically align the attributes.

If *compute_entropy* is set to True it computes the entropy of each cluster of attributes that can be used to further improve the meta-blocking performance.

*target_threshold* parameter regulates the similarity that the values of the attributes should have to be clustered together

In [123]:
clusters = sparker.AttributeClustering.cluster_similar_attributes(profiles,
                                  num_hashes=128,
                                  target_threshold=0.5,
                                  compute_entropy=True)
clusters

[{'cluster_id': 0, 'keys': ['0_Birthdate', '1_Birthdate'], 'entropy': 5.693867955071256},
 {'cluster_id': 1, 'keys': ['0_DisplayName', '1_DisplayName'], 'entropy': 10.224024572751448},
 {'cluster_id': 2, 'keys': ['1_address', '0_State', '1_City', '1_Country', '0_PhoneNumber', '0_address', '1_State', '1_PhoneNumber', '0_City', '0_Country'], 'entropy': 27.15030253011959}]

## Blocking
Now we can perform blocking by using the generated clusters

In [124]:
blocks = sparker.Blocking.create_blocks_clusters(profiles, clusters, separator_ids)
print("Number of blocks",blocks.count())

Number of blocks 1958


In [125]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [126]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.005)

In [127]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [21]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 1.0
Precision 0.08130081300813008
Number of comparisons 123


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.


For every partition of the RDD, the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [128]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [129]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

block_entropies = sc.broadcast(blocks.map(lambda b : (b.block_id, b.entropy)).collectAsMap())

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
# gt_broadcast = sc.broadcast(new_gt)

### Meta-blocking with BLAST
BLAST employs $\chi^2$ weighting scheme.

*chi2divider* parameter regulates the pruning aggressivity, a lower value performs a more aggressive pruning.

In [130]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CHI_SQUARE,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          use_entropy=True,
                          blocks_entropies=block_entropies,
                          chi2divider=2.0
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 1754897


## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [131]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(0, 21670, 672420.7616024732),
 (9060, 21504, 913.3945313226179),
 (9060, 19457, 913.3945313226179),
 (9060, 13824, 544.0640053886309),
 (9060, 15373, 913.3945313226179),
 (9060, 17934, 913.3945313226179),
 (9060, 19990, 913.3945313226179),
 (9060, 15896, 913.3945313226179),
 (9060, 14361, 6147.458988700017),
 (9060, 15899, 913.3945313226179)]

In [132]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [133]:
max_by_group.take(60)

[(0, 21670, 672420.7616024733),
 (9060, 14361, 6147.458988700017),
 (888, 16125, 10277.622530509625),
 (16, 21686, 566968.986575489),
 (32, 21688, 18170.873395867482),
 (44, 21685, 15316.42500030979),
 (48, 21682, 21269.562558138874),
 (52, 21685, 15316.42500030979),
 (64, 21685, 13348.080702784726),
 (72, 21558, 5854.720093601698),
 (80, 21671, 17661.135083382203),
 (5608, 15417, 10277.622530509625),
 (2060, 19723, 3297.2435451800097),
 (1692, 21671, 6348.70055589886),
 (3884, 18433, 4355.376354079065),
 (4284, 20768, 3254.684804367526),
 (5316, 15537, 10277.622530509625),
 (12236, 18165, 3475.270703178818),
 (9424, 21043, 2469.099321016459),
 (4456, 20768, 2594.2825612760607),
 (12524, 20185, 10277.622530509625),
 (5872, 21043, 3297.2435451800097),
 (884, 16480, 2469.099321016459),
 (11380, 21043, 4953.534640815427),
 (2300, 14657, 5114.922239782524),
 (7172, 18695, 13717.661434241369),
 (2440, 16672, 13717.661434241369),
 (3864, 18273, 19285.077052799184),
 (5400, 18026, 10277.62253

In [165]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0      1              2
0          0  21670  672420.761602
1       9060  14361    6147.458989
2        888  16125   10277.622531
3         16  21686  566968.986575
4         32  21688   18170.873396
...      ...    ...            ...
13139  10183  13249    3297.243545
13140  10247  16668   13717.661434
13141  10323  21369   10277.622531
13142  10363  19475    3297.243545
13143  10415  21226    3297.243545

[13144 rows x 3 columns]

In [166]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id   N_id         weight
0     0  21670  672420.761602
1  9060  14361    6147.458989
2   888  16125   10277.622531
3    16  21686  566968.986575
4    32  21688   18170.873396

In [167]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id   N_id         weight
0          0  21670  672420.761602
1       9060  14361    6147.458989
2        888  16125   10277.622531
3         16  21686  566968.986575
4         32  21688   18170.873396
...      ...    ...            ...
13139  10183  13249    3297.243545
13140  10247  16668   13717.661434
13141  10323  21369   10277.622531
13142  10363  19475    3297.243545
13143  10415  21226    3297.243545

[13144 rows x 3 columns]

In [168]:
max1 = n["weight"].max()
max1 = round(max1,0)
max1

777873.0

In [169]:
max_percentage = (max1*30)/100


n["N_id"] = n["N_id"]-(separator_id+1)
new_df = n[(n['weight']>max_percentage)]
new_df

P_id  N_id         weight
0         0  8526  672420.761602
3        16  8542  566968.986575
35        4  8530  562043.043181
36        8  8534  777872.536629
41       12  8538  514243.099062
3291      9  8535  566968.986575
3328     13  8539  566968.986575
3433      5  8531  514243.099062
3562     17  8543  619694.874089
6577     14  8540  566968.986575
6610      2  8528  404953.961672
6611      6  8532  672420.761602
6612     10  8536  541761.421249
6636     18  8544  514243.099062
9862     15  8541  619694.874089
9897      3  8529  562043.043181
9898      7  8533  672420.761602
9915     11  8537  514243.099062
10169    19  8545  514243.099062

In [170]:
new_df["c_id"] = [x for x in range(0,len(new_df))]
new_df

/tmp/ipykernel_5828/1762727809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["c_id"] = [x for x in range(0,len(new_df))]


P_id  N_id         weight  c_id
0         0  8526  672420.761602     0
3        16  8542  566968.986575     1
35        4  8530  562043.043181     2
36        8  8534  777872.536629     3
41       12  8538  514243.099062     4
3291      9  8535  566968.986575     5
3328     13  8539  566968.986575     6
3433      5  8531  514243.099062     7
3562     17  8543  619694.874089     8
6577     14  8540  566968.986575     9
6610      2  8528  404953.961672    10
6611      6  8532  672420.761602    11
6612     10  8536  541761.421249    12
6636     18  8544  514243.099062    13
9862     15  8541  619694.874089    14
9897      3  8529  562043.043181    15
9898      7  8533  672420.761602    16
9915     11  8537  514243.099062    17
10169    19  8545  514243.099062    18

In [171]:
col = ["P_id","N_id","weight","cluster_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------------------+----------+
|P_id|N_id|            weight|cluster_id|
+----+----+------------------+----------+
|   0|8526| 672420.7616024733|         0|
|  16|8542|  566968.986575489|         1|
|   4|8530| 562043.0431813973|         2|
|   8|8534| 777872.5366294578|         3|
|  12|8538| 514243.0990619967|         4|
|   9|8535|  566968.986575489|         5|
|  13|8539|  566968.986575489|         6|
|   5|8531| 514243.0990619967|         7|
|  17|8543| 619694.8740889812|         8|
|  14|8540|  566968.986575489|         9|
|   2|8528|404953.96167194814|        10|
|   6|8532| 672420.7616024733|        11|
|  10|8536|  541761.421248922|        12|
|  18|8544| 514243.0990619967|        13|
|  15|8541| 619694.8740889813|        14|
|   3|8529| 562043.0431813973|        15|
|   7|8533| 672420.7616024733|        16|
|  11|8537| 514243.0990619967|        17|
|  19|8545| 514243.0990619967|        18|
+----+----+------------------+----------+



In [172]:
ss = SparkSession(sc)


In [173]:
# df1 = ss.read.option("header",True)\
#         .option("inferSchema",True)\
#         .csv('hdfs://localhost:9000/Final_Project/sample/Sample1_cleaned.csv')
# df1.show(5)

In [174]:
# df2 = ss.read.option("header",True)\
#         .option("inferSchema",True)\
#         .csv('file:///home/aditi/BigData/AAI_Project/dataset/Sample2_cleaned.csv')
# df2.show(5)

In [175]:
from pyspark.sql.functions import monotonically_increasing_id

df1 = df1.withColumn("new_id", monotonically_increasing_id())
df2 = df2.withColumn("new_id", monotonically_increasing_id())

df1.show(5)
df2.show(5)



common_df = df1.join(df_weight,df1.new_id == df_weight.P_id,"inner")
common_N = df2.join(df_weight,df2.new_id == df_weight.N_id,"inner")


############################ union dataframe
import functools

def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)


# unionAll
result3 = unionAll([common_df, common_N])
result3.show()

final = result3.toPandas()






+----------+----+-------+--------------------+---+-------------+--------------+--------------------+------+
| Birthdate|City|Country|         DisplayName| ID|  PhoneNumber|         State|             address|new_id|
+----------+----+-------+--------------------+---+-------------+--------------+--------------------+------+
|2000-04-20|null|  India|    Dolly Chaurasiya|  0|+917703901236|Andhra Pradesh|Ap #324-5397 Semp...|     0|
|1990-09-03|null|  India|Patnaik Priyadars...|  1|+918939498995|Andhra Pradesh|Opposite Winterco...|     1|
|1993-03-20|null|  India|   Sushaant Valmikei|  2|+917881007998|Andhra Pradesh|P.O. Box 861,Near...|     2|
|1978-10-03|null|  India|        Shiva Sonkar|  3|+919910914898|Andhra Pradesh|P.O. Box 801, 969...|     3|
|1970-06-15|null|  India|     Janardan Desuza|  4|+918839083491|Andhra Pradesh|P.O. Box 541, 912...|     4|
+----------+----+-------+--------------------+---+-------------+--------------+--------------------+------+
only showing top 5 rows

+--

In [176]:
final = final.drop(["P_id","N_id","weight"], axis=1)
final.head()

final_cluster = final.groupby(['cluster_id','new_id'])
final_cluster.first()

Birthdate  City Country          DisplayName  ID  \
cluster_id new_id                                                      
0          0       2000-04-20  None   India     Dolly Chaurasiya   0   
           8526    2000-04-20  None   India     Dolly Chaurasiya   0   
1          16      1985-09-12  None   India      Bhawna Mazumdar  16   
           8542    1985-09-12  None   India      Bhawna Mazumdar  16   
2          4       1970-06-15  None   India      Janardan Desuza   4   
           8530    1970-06-15  None   India       Janardan Dixit   4   
3          8       1997-06-15  None   India        Aniruddh Bhat   8   
           8534    1997-06-15  None   India        Aniruddh Bhat   8   
4          12      1991-09-18  None   India      Mudita Tripathi  12   
           8538    1991-09-18  None   India      Mudita Tripathi  12   
5          9       1986-04-02  None   India         Vishal Patil   9   
           8535    1986-04-02  None   India         Vishal Patil   9   
6          13      1975-12-20  None   India       Jagannath Bhat  13   
           8539    1975-12-20  None   India       Jagannath Bhat  13   
7          5       1987-09-08  None   India         Trisha Dhoni   5   
           8531    1987-09-08  None   India         Trisha Dhoni   5   
8          17      1991-03-19  None   India         Sarthak Umar  17   
           8543    1991-03-19  None   India         Sarthak Umar  17   
9          14      1982-10-04  None   India        Ramnik Saxena  14   
           8540    1982-10-04  None   India        Ramnik Saxena  14   
10         2       1993-03-20  None   India    Sushaant Valmikei   2   
           8528    1993-03-20  None   India      Sushant Valmiki   2   
11         6       1997-08-30  None   India  Sacchidanand Prasad   6   
           8532    1997-08-30  None   India  Sacchidanand Prasad   6   
12         10      1976-04-01  None   India              Shreyaa  10   
           8536    1976-04-01  None   India           Shreya Raj  10   
13         18      1986-04-13  None   India      Minakshi Jindal  18   
           8544    1986-04-13  None   India      Minakshi Jindal  18   
14         15      1998-03-13  None   India         Maitri Goyal  15   
           8541    1998-03-13  None   India         Maitri Goyal  15   
15         3       1978-10-03  None   India         Shiva Sonkar   3   
           8529    1978-10-03  None   India   Shivshankar Sonkar   3   
16         7       1983-04-05  None   India        Jwala Chouhan   7   
           8533    1983-04-05  None   India        Jwala Chouhan   7   
17         11      1971-03-15  None   India    Ambika Khandelwal  11   
           8537    1971-03-15  None   India    Ambika Khandelwal  11   
18         19      1999-01-03  None   India         Uddhav Bhati  19   
           8545    1999-01-03  None   India         Uddhav Bhati  19   

                     PhoneNumber           State  \
cluster_id new_id                                  
0          0       +917703901236  Andhra Pradesh   
           8526    +917703901236  Andhra Pradesh   
1          16      +917815270286  Andhra Pradesh   
           8542    +917815270286  Andhra Pradesh   
2          4       +918839083491  Andhra Pradesh   
           8530    +918839083491  Andhra Pradesh   
3          8       +916846051804  Andhra Pradesh   
           8534    +916846051804  Andhra Pradesh   
4          12      +918970494925  Andhra Pradesh   
           8538    +918970494925  Andhra Pradesh   
5          9       +918020258041  Andhra Pradesh   
           8535    +918020258041  Andhra Pradesh   
6          13      +915523170584  Andhra Pradesh   
           8539    +915523170584  Andhra Pradesh   
7          5       +916857918837  Andhra Pradesh   
           8531    +916857918837  Andhra Pradesh   
8          17      +917810211752  Andhra Pradesh   
           8543    +917810211752  Andhra Pradesh   
9          14      +913426438028  Andhra Pradesh   
           8540    +913426438028  Andhra Prades

In [177]:
df = pd.concat(map(lambda x: x[1], final_cluster))

df = df.reset_index().drop(["index","new_id","ID"],axis =1)

new_col = [x for x in range(0,len(df))]
df.insert(loc=0, column='id', value=new_col)
#df.to_json("/home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/Dataset/data/output")


In [178]:
df

id   Birthdate  City Country          DisplayName    PhoneNumber  \
0    0  2000-04-20  None   India     Dolly Chaurasiya  +917703901236   
1    1  2000-04-20  None   India     Dolly Chaurasiya  +917703901236   
2    2  1985-09-12  None   India      Bhawna Mazumdar  +917815270286   
3    3  1985-09-12  None   India      Bhawna Mazumdar  +917815270286   
4    4  1970-06-15  None   India      Janardan Desuza  +918839083491   
5    5  1970-06-15  None   India       Janardan Dixit  +918839083491   
6    6  1997-06-15  None   India        Aniruddh Bhat  +916846051804   
7    7  1997-06-15  None   India        Aniruddh Bhat  +916846051804   
8    8  1991-09-18  None   India      Mudita Tripathi  +918970494925   
9    9  1991-09-18  None   India      Mudita Tripathi  +918970494925   
10  10  1986-04-02  None   India         Vishal Patil  +918020258041   
11  11  1986-04-02  None   India         Vishal Patil  +918020258041   
12  12  1975-12-20  None   India       Jagannath Bhat  +915523170584   
13  13  1975-12-20  None   India       Jagannath Bhat  +915523170584   
14  14  1987-09-08  None   India         Trisha Dhoni  +916857918837   
15  15  1987-09-08  None   India         Trisha Dhoni  +916857918837   
16  16  1991-03-19  None   India         Sarthak Umar  +917810211752   
17  17  1991-03-19  None   India         Sarthak Umar  +917810211752   
18  18  1982-10-04  None   India        Ramnik Saxena  +913426438028   
19  19  1982-10-04  None   India        Ramnik Saxena  +913426438028   
20  20  1993-03-20  None   India    Sushaant Valmikei  +917881007998   
21  21  1993-03-20  None   India      Sushant Valmiki  +917881007998   
22  22  1997-08-30  None   India  Sacchidanand Prasad  +917317068155   
23  23  1997-08-30  None   India  Sacchidanand Prasad  +917317068155   
24  24  1976-04-01  None   India              Shreyaa  +918052414788   
25  25  1976-04-01  None   India           Shreya Raj  +918052414788   
26  26  1986-04-13  None   India      Minakshi Jindal  +916848926412   
27  27  1986-04-13  None   India      Minakshi Jindal  +916848926412   
28  28  1998-03-13  None   India         Maitri Goyal  +918939475117   
29  29  1998-03-13  None   India         Maitri Goyal  +918939475117   
30  30  1978-10-03  None   India         Shiva Sonkar  +919910914898   
31  31  1978-10-03  None   India   Shivshankar Sonkar  +919910914898   
32  32  1983-04-05  None   India        Jwala Chouhan  +917881891623   
33  33  1983-04-05  None   India        Jwala Chouhan  +917881891623   
34  34  1971-03-15  None   India    Ambika Khandelwal  +912717363618   
35  35  1971-03-15  None   India    Ambika Khandelwal  +912717363618   
36  36  1999-01-03  None   India         Uddhav Bhati  +915654053729   
37  37  1999-01-03  None   India         Uddhav Bhati  +915654053729   

             State                                            address  \
0   Andhra Pradesh  Ap #324-5397 Semper Av.,Opposite Wildemarsh,A ...   
1   Andhra Pradesh  Ap #324-5397 Semper Av.,Opposite Wildemarsh,A ...   
2   Andhra Pradesh  Ap #276-9059 Luctus Rd.,Opp. Esterash,Aravinda...   
3   Andhra Pradesh  Ap #276-9059 Luctus Rd.,Opp. Esterash,Aravinda...   
4   Andhra Pradesh  P.O. Box 541, 9122 Non, St.,Opposite Fairsage,...   
5   Andhra Pradesh  P.O. Box 541, 9122 Non, St.,Opposite Fairsage,...   
6   Andhra Pradesh  P.O. Box 957, 2070 Eu Ave,Opposite Aelhurst,An...   
7   Andhra Pradesh  P.O. Box 957, 2070 Eu Ave,Opposite Aelhurst,An...   
8   Andhra Pradesh  3437 Commodo St.,Near Springfox,Appecharla,ANA...   
9   Andhra Pradesh  3437 Commodo St.,Near Springfox,Appecharla,ANA...   
10  Andhra Pradesh  Ap #106-9892 Non, Avenue,Near Wooddell,Anantap...   
11  Andhra Pradesh  Ap #106-9892 Non, Avenue,Near Wooddell,Anantap...   
12  Andhra Pradesh  938-165 Vitae Avenue,Opp. Lightkeep,Appilepall...   
13  Andhra Pradesh  938-165 Vitae Avenue,Opp. Lightkeep,Appilepall...   
14  Andhra Pradesh  436-9051 Lorem. Av.,Opposite Shadowmaple,Amidy...   
15  Andhra Pradesh  436-905